# 02 – Résultats d'entraînement DermaAI

Ce notebook affiche :

* Les courbes d'apprentissage (loss / accuracy) des deux phases
* La matrice de confusion sur le jeu de test
* Le rapport de classification par classe

In [ ]:
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

# Ajouter src/ au path pour importer data_loader et predict
src_dir = Path("..") / "src"
if str(src_dir.resolve()) not in sys.path:
    sys.path.insert(0, str(src_dir.resolve()))

CLASSES = ["akiec", "bcc", "bkl", "df", "mel", "nv", "vasc"]
CLASS_LABELS = {
    "akiec": "Actinic keratoses",
    "bcc":   "Basal cell carcinoma",
    "bkl":   "Benign keratosis-like",
    "df":    "Dermatofibroma",
    "mel":   "Melanoma",
    "nv":    "Melanocytic nevi",
    "vasc":  "Vascular lesions",
}

base_dir   = Path("..")
model_path = base_dir / "models" / "dermai_cnn.h5"
splits_dir = base_dir / "data" / "splits"

print("Modèle :",  model_path.resolve())
print("Splits :",  splits_dir.resolve())

## Courbes d'apprentissage

> **Note** : les courbes ci-dessous sont générées à partir de l'historique
> retourné par `train.py`. Si vous avez déjà lancé l'entraînement et sauvegardé
> l'historique (`.npy` ou `.csv`), chargez-le ici. Dans le cas contraire, une
> courbe simulée est affichée à des fins d'illustration.

In [ ]:
history_path = base_dir / "models" / "history.npy"

if history_path.exists():
    history = np.load(str(history_path), allow_pickle=True).item()
else:
    print("Historique introuvable – affichage d'une courbe simulée.")
    np.random.seed(42)
    n = 20
    history = {
        "loss":         np.linspace(1.8, 0.4, n) + np.random.normal(0, 0.05, n),
        "val_loss":     np.linspace(1.9, 0.5, n) + np.random.normal(0, 0.07, n),
        "accuracy":     np.linspace(0.3, 0.87, n) + np.random.normal(0, 0.02, n),
        "val_accuracy": np.linspace(0.28, 0.84, n) + np.random.normal(0, 0.03, n),
    }

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history["loss"],     label="Train loss")
axes[0].plot(history["val_loss"], label="Val loss", linestyle="--")
axes[0].set_title("Courbe de perte (loss)")
axes[0].set_xlabel("Époque")
axes[0].set_ylabel("Loss")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history["accuracy"],     label="Train accuracy")
axes[1].plot(history["val_accuracy"], label="Val accuracy", linestyle="--")
axes[1].set_title("Courbe de précision (accuracy)")
axes[1].set_xlabel("Époque")
axes[1].set_ylabel("Accuracy")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Matrice de confusion

In [ ]:
import tensorflow as tf

if model_path.exists() and (splits_dir / "test.csv").exists():
    from data_loader import load_split, CLASS_TO_IDX

    model = tf.keras.models.load_model(str(model_path))
    test_ds = load_split(str(splits_dir / "test.csv"), batch_size=32)

    y_true, y_pred = [], []
    for imgs, labels in test_ds:
        preds = model.predict(imgs, verbose=0)
        y_true.extend(labels.numpy())
        y_pred.extend(np.argmax(preds, axis=1))

    cm = confusion_matrix(y_true, y_pred)
else:
    print("Modèle ou splits introuvables – matrice de confusion simulée.")
    np.random.seed(0)
    cm = np.random.randint(0, 50, (7, 7))
    np.fill_diagonal(cm, np.random.randint(150, 300, 7))
    y_true = [0] * 10  # bidon pour le rapport ci-dessous
    y_pred = [0] * 10

fig, ax = plt.subplots(figsize=(9, 7))
sns.heatmap(
    cm, annot=True, fmt='d', cmap='Blues',
    xticklabels=CLASSES, yticklabels=CLASSES, ax=ax
)
ax.set_title("Matrice de confusion – jeu de test")
ax.set_xlabel("Classe prédite")
ax.set_ylabel("Classe réelle")
plt.tight_layout()
plt.show()

## Rapport de classification

In [ ]:
if len(set(y_true)) > 1:
    print(classification_report(y_true, y_pred, target_names=CLASSES))
else:
    print("Rapport non disponible (données simulées).")